In [1]:
import numpy as np
import torch

In [2]:
import sys
!{sys.executable} -m pip install -e ../.
from privacy_meter.audit import Audit
from privacy_meter.dataset import Dataset
from privacy_meter.hypothesis_test import threshold_func
from privacy_meter.information_source import InformationSource
from privacy_meter.information_source_signal import ModelLoss
from privacy_meter.model import PytorchModel

Obtaining file:///Users/aadyaamaddi/Desktop/ML%20Privacy%20Meter/privacy_meter
  Preparing metadata (setup.py) ... done
  Attempting uninstall: privacy-meter
    Found existing installation: privacy-meter 1.0
    Uninstalling privacy-meter-1.0:
      Successfully uninstalled privacy-meter-1.0
  Running setup.py develop for privacy-meter


In [3]:
from transformers import BertForMaskedLM, BertTokenizer
from datasets import load_dataset

In [23]:
class MaskedLMDataset(Dataset):
    def __init__(
        self,
        data_dict: dict,
        default_input: str = 'masked_input',
        default_output: str = 'output',
        preproc_fn_dict: dict = None,
        tokenizer = None
    ):  
        super().__init__(
            data_dict=data_dict,
            default_input=default_input,
            default_output=default_output,
            preproc_fn_dict=preproc_fn_dict
        )
        
        self.tokenizer = tokenizer

    def tokenize_batch(self, batch):
        tokenized_batch = []
        for sentence in batch:
            tokenized_text = self.tokenizer.tokenize(sentence)
            tokenized_batch.append(self.tokenizer.convert_tokens_to_ids(tokenized_text))
        return tokenized_batch

    def untokenize_batch(self, batch):
        return [self.tokenizer.convert_ids_to_tokens(sentence) for sentence in batch]

In [24]:
# download dataset from huggingface
hf_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
hf_dataset.set_format('numpy')
hf_dataset.keys()

Reusing dataset wikitext (/Users/aadyaamaddi/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


  0%|          | 0/3 [00:00<?, ?it/s]

dict_keys(['test', 'train', 'validation'])

In [25]:
train_ds = {
    'masked_input': hf_dataset['train']['text'],
    'output': hf_dataset['train']['text']
}

test_ds = {
    'masked_input': hf_dataset['test']['text'],
    'output': hf_dataset['test']['text']
}

validation_ds = {
    'masked_input': hf_dataset['validation']['text'],
    'output': hf_dataset['validation']['text']
}

In [26]:
data_dict = {
    'train': train_ds, 
    'test': test_ds,
    'validation': validation_ds
}

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
masked_lm_dataset = MaskedLMDataset(
    data_dict=data_dict,
    default_input='masked_input',
    default_output='output',
    tokenizer=tokenizer
)

In [42]:
batch = masked_lm_dataset.get_feature(split_name='train', feature_name='masked_input', indices=[1])
tokenized_batch = masked_lm_dataset.tokenize_batch(batch)

In [49]:
tokenized_tensor = torch.Tensor(tokenized_batch)
segment_tensor = torch.Tensor([[0] * len(x) for x in tokenized_batch])

In [50]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [52]:
predictions = model(tokenized_tensor, segment_tensor)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)